Sample code for variational Monte Carlo method for helium 4

In [ ]:
import numpy as np # library for numerics
import scipy.linalg as la # library for linear algebra
import scipy.sparse.linalg as sla # library for sparse linear algebra
from scipy.sparse import csr_matrix, csc_matrix, coo_matrix, lil_matrix
import random as rd # library for random numbers
import matplotlib.pyplot as plt # library for plot

Class for a variational wave function for helium 4 by W. L. McMillan, in Phys. Rev. 138, A442 (1965):

$\psi (\vec{r}_1 , \vec{r}_2 , \dots , \vec{r}_N)
=
\prod_{i<j; i,j=1}^{N}f(r_{ij})$, where $f(r)=\exp [-u (r)]$ and $u(r) = (a_1/r)^{a_2}$

In [ ]:
class helium4:

 # constructor
  def __init__(self,L,N,a1,a2):
    self.L = L # lnear length of cubic
    self.N = N # particle number
    self.a1 = a1 # variational parameter defines length scale
    self.a2 = a2 # power
    self.rcut = 0.5*a1 # cutoff for particle-particle distance
    self.config = np.zeros((N,3),dtype=np.float64) # real space particle configuration
    self.dist = np.zeros((N,N),dtype=np.float64) # two-particle distance
 
 # u(r)
  def function_u(self,r):
    if r > self.rcut:
      uofr = (self.a1/r)**self.a2
    else:
      uofr = (self.a1/self.rcut)**self.a2
    return uofr

# initialize config and dist
  def initialize_config(self):
    for j in range(self.N):
      for i in range(3):
        self.config[j][i] = self.L * rd.random()
    for iupdate in range(self.N):
      self.update_dist(iupdate)

# update position of the iupdate th particle in config
  def update_config(self,iupdate):
    d = 0.05*self.L
    for i in range(3):
      self.config[iupdate][i] += 2.0*d*(rd.random()-0.5)
      if self.config[iupdate][i] > self.L:
        self.config[iupdate][i] -= self.L
      elif self.config[iupdate][i] < 0.0:
        self.config[iupdate][i] += self.L

# update distance between the iupdate th particle and others in dist
  def update_dist(self,iupdate):
    for j in range(self.N):
      tmp_dist = self.dist_periodic(he4.config[j],he4.config[iupdate]) 
      if tmp_dist > self.rcut:
        self.dist[j,iupdate] = tmp_dist
        self.dist[iupdate,j] = tmp_dist
      else:
        self.dist[j,iupdate] = self.rcut
        self.dist[iupdate,j] = self.rcut

# return shortest distance between two particles (where their position is given by vec1, vec2) in 3D torus
  def dist_periodic(self,vec1,vec2): 
    dist_old = np.sqrt(3.0)*self.L
    for iz in range(-1,2):
      for iy in range(-1,2):
        for ix in range(-1,2):
          tmp_dist = 0.0
          tmp_dist += ( self.L*ix + vec1[0] - vec2[0])**2
          tmp_dist += ( self.L*iy + vec1[1] - vec2[1])**2
          tmp_dist += ( self.L*iz + vec1[2] - vec2[2])**2
          tmp_dist = np.sqrt(tmp_dist)
          if tmp_dist < dist_old:
            dist_old = tmp_dist
    return dist_old

# accumulate samples for two point distribution function
  def accum_Nrdr(self,Ndr,Nrdr): 
    rmax = 0.5*self.L
    dr = rmax/Ndr
    for m in range(Ndr):
      r = rmax*(m + 0.5)/Ndr
      for i in range(self.N-1):
        for j in range(i+1,self.N):
          if self.dist[i][j] >= r - 0.5*dr and self.dist[i][j] < r + 0.5*dr:
            Nrdr[m] = Nrdr[m] + 1.0

# calculate two particle distribution function (see Lecture No.7)
  def calc_gofr(self,Nsample,Ndr,Nrdr,gofr,vecr): 
    Omega = self.L**3
    rho = 1.0*self.N / Omega
    rmax = 0.5*self.L
    dr = rmax/Ndr
    for m in  range(Ndr):
      r = rmax*(m+0.5)/Ndr
      vecr[m] = r
      gofr[m] = Nrdr[m]/(2.0*Nsample*np.pi*rho*rho*Omega*r*r*dr)


Complte the following MC sampling by Metropolis algorithm with probability $P(\vec{r}_1 , \vec{r}_2 , \dots , \vec{r}_N) \propto |\psi (\vec{r}_1 , \vec{r}_2 , \dots , \vec{r}_N)|^2$.

In [ ]:
rd.seed(10) # fix the seed for random number generation
# construct McMillan's wave function: N=32 particles in L^3 3D torus (L=11.2 angstrom) and a1=2.6 angstrom and a2=5
he4 = helium4(11.2,32,2.6,5)
he4.initialize_config()
# prepare arrays for calculation of g(r) 
Ndr = 64
Nrdr = np.zeros((Ndr),dtype=np.float64)
gofr = np.zeros((Ndr),dtype=np.float64) # vector for g(r)
vecr = np.zeros((Ndr),dtype=np.float64) # vector for r
# Nwup: number of warmup/brun-in steps
Nwup = 400
# Nsample: number of MC steps
Nsample = 3200
# preparation for Metropolis update
config_old = np.zeros((he4.N,3),dtype=np.float64)
dist_old = np.zeros((he4.N,he4.N),dtype=np.float64)
for i in range(Nwup):
  for j in range(he4.N):
    config_old[:][:] = he4.config[:][:]
    dist_old[:][:] = he4.dist[:][:]
    # choose a particle to be updated
    iupdate = rd.randrange(he4.N)
    # update the position of the iupdate th particle
    he4.update_config(iupdate)
    he4.update_dist(iupdate)
    # begin Metropolis
    # end Metropolis
  if i%100 == 0:
    print(i,"the step for warmup")

for i in range(Nsample):
  for j in range(he4.N):
    config_old[:][:] = he4.config[:][:]
    dist_old[:][:] = he4.dist[:][:]
    # choose a particle to be updated
    iupdate = rd.randrange(he4.N)
    # update the position of the iupdate th particle
    he4.update_config(iupdate)
    he4.update_dist(iupdate)
    # begin Metropolis
    # begin Metropolis
  if i%100 == 0:
    print(i,"the MC step")
#  he4.accum_Nrdr(Ndr,Nrdr)
#he4.calc_gofr(Nsample,Ndr,Nrdr,gofr,vecr)
#plt.plot(vecr,gofr)
#for i in range(Ndr):
#  print(vecr[i]," ",gofr[i])